## 페이지 및 pdf 크롤링

In [ ]:
pip install requests beautifulsoup4 tqdm pdfminer.six pymupdf

### 라이브러리

In [4]:
import os
import re
import time
from urllib.parse import urljoin
import requests
import pandas as pd
import fitz
from bs4 import BeautifulSoup
from tqdm import tqdm
import random

### 고정 변수 할당

In [2]:
BASE = "https://finance.naver.com"

session = requests.Session()
session.headers.update({
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
    "Accept-Language": "ko-KR,ko;q=0.9,en-US;q=0.7,en;q=0.6",
    "Connection": "keep-alive",
    "Referer": BASE,
})


In [3]:
BASE_DIR = r"C:\Users\User\Desktop\kdt\pjt_bok_1"
PDF_DIR = os.path.join(BASE_DIR, "pdf")
TXT_DIR = os.path.join(BASE_DIR, "text")
DEBUG_DIR = os.path.join(BASE_DIR, "debug_html")
os.makedirs(PDF_DIR, exist_ok=True)
os.makedirs(TXT_DIR, exist_ok=True)
os.makedirs(DEBUG_DIR, exist_ok=True)


In [ ]:
BROKER_CODE = "18"
DATE_FROM = "2012-01-01" # 필요시 수정
DATE_TO = "2025-12-31" # 필요시 수정
SEARCH_TYPE = "writeDate"
SLEEP_SEC = 0.3
MAX_PAGE = 300 # 필요시 수정


### 세션 열기 함수

In [ ]:
DEFAULT_HEADERS = {
    "User-Agent": "Mozilla/5.0",
}

def fetch_html(url: str, referer: str | None = None, max_retries: int = 6) -> str:
    headers = DEFAULT_HEADERS.copy()
    if referer:
        headers["Referer"] = referer

    last_err = None

    for attempt in range(1, max_retries + 1):
        try:
            r = session.get(
                url,
                headers=headers,
                timeout=(10, 25),   # (connect, read)
                allow_redirects=True,
            )
            r.raise_for_status()

            r.encoding = r.apparent_encoding
            return r.text

        except (requests.exceptions.ConnectionError,
                requests.exceptions.Timeout,
                requests.exceptions.ChunkedEncodingError,
                requests.exceptions.HTTPError) as e:
            last_err = e

            sleep_sec = min(60, (2 ** attempt)) + random.uniform(0.2, 1.2)
            time.sleep(sleep_sec)

    raise last_err


In [6]:
def fetch_html(url: str, referer: str | None = None) -> str:
    headers = {
        "User-Agent": "Mozilla/5.0"
    }
    if referer:
        headers["Referer"] = referer

    r = session.get(
        url,
        headers=headers,
        timeout=25,
        allow_redirects=True,
    )
    r.raise_for_status()

    r.encoding = r.apparent_encoding
    return r.text


### 페이지 url 리스트 받는 함수

In [7]:
def get_list_url(page: int) -> str:
    return (
        f"{BASE}/research/debenture_list.naver?"
        f"keyword=&brokerCode={BROKER_CODE}"
        f"&searchType={SEARCH_TYPE}"
        f"&writeFromDate={DATE_FROM}&writeToDate={DATE_TO}"
        f"&x=0&y=0&page={page}"
    )


### nid(고유키) 받는 함수

In [ ]:
nids = set()
def get_list_nids(page: int) -> list[int]:
    url = get_list_url(page)
    html = fetch_html(url)
    soup = BeautifulSoup(html, "lxml")

    nids = []
    for a in soup.select('a[href*="debenture_read.naver?nid="]'):
        href = a.get("href", "")
        m = re.search(r"nid=(\d+)", href)
        if m:
            nids.append(int(m.group(1)))

    uniq, seen = [], set()
    for x in nids:
        if x not in seen:
            uniq.append(x)
            seen.add(x)
    return uniq

### 상세페이지 parse 하기 함수

In [9]:
def parse_detail_page_raw(nid: int, debug: bool = False) -> dict | None:
    list_url = get_list_url(1)
    detail_url = f"{BASE}/research/debenture_read.naver?nid={nid}"

    html = fetch_html(detail_url, referer=list_url)
    soup = BeautifulSoup(html, "lxml")
    text_all = soup.get_text(" ", strip=True)

    og = soup.select_one('meta[property="og:title"]')
    title = og.get("content").strip() if og and og.get("content") else None

    source_p = soup.select_one("p.source")
    source_raw = source_p.get_text(" ", strip=True) if source_p else None

    date = None
    org = None
    if source_raw:
        m_date = re.search(r"(20\d{2}\.\d{2}\.\d{2})", source_raw)
        if m_date:
            date = m_date.group(1)
            org = source_raw.split(date)[0].replace("|", " ").strip()
            org = re.sub(r"\s+", " ", org)

    page_text = None
    td_node = soup.select_one("#contentarea_left td.view_cnt")
    if td_node:
        ps = td_node.select("p")
        if ps:
            page_text = "\n".join(
                p.get_text(" ", strip=True) for p in ps if p.get_text(strip=True)
            )
        else:
            page_text = td_node.get_text("\n", strip=True)

    if not page_text:
        selector = "#contentarea_left > div.box_type_m.box_type_m3 > table > tbody > tr:nth-child(3) > td > div:nth-child(1) > p"
        p_node = soup.select_one(selector)
        if p_node:
            page_text = p_node.get_text("\n", strip=True)

    if page_text:
        page_text = re.sub(r"\n{3,}", "\n\n", page_text).strip()

    if title == "네이버페이 증권" and not date and not page_text:
        if debug:
            with open(os.path.join(DEBUG_DIR, f"landing_{nid}.html"), "w", encoding="utf-8") as f:
                f.write(html)
        return None

    if not (title or source_raw or page_text):
        if debug:
            with open(os.path.join(DEBUG_DIR, f"parse_fail_{nid}.html"), "w", encoding="utf-8") as f:
                f.write(html)
        return None

    pdf_url = None
    for a in soup.select("a[href]"):
        if "btn_report.gif" in str(a):
            pdf_url = urljoin(BASE, a.get("href"))
            break

    return {
        "nid": nid,
        "title": title,
        "org": org,
        "date": date,
        "source_raw": source_raw,
        "page_text": page_text,
        "pdf_url": pdf_url,
    }


### 파일 이름 안정화 함수

In [10]:
def safe_filename(s: str, maxlen: int = 180) -> str:
    s = re.sub(r"[\\/:*?\"<>|]", "_", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s[:maxlen]


### pdf 다운로드 함수

In [18]:
def download_pdf(meta: dict) -> str:
    filename = f"{meta['date']}_{meta.get('org') or 'UNKNOWN'}_{meta['nid']}.pdf"
    filename = safe_filename(filename)
    path = os.path.join(PDF_DIR, filename)

    if os.path.exists(path) and os.path.getsize(path) > 0:
        return path

    r = session.get(meta["pdf_url"], timeout=60)
    r.raise_for_status()
    with open(path, "wb") as f:
        f.write(r.content)
    return path


### pdf에서 text 추출 함수

In [19]:
def extract_text_from_pdf(pdf_path: str) -> str:
    doc = fitz.open(pdf_path)
    text = "\n".join(page.get_text() for page in doc)
    doc.close()
    return text


### pdf text를 csv 파일에 저장

In [20]:
def save_pdf_text(meta: dict, pdf_path: str, pdf_text: str) -> str:
    txt_name = os.path.basename(pdf_path).replace(".pdf", ".txt")
    txt_path = os.path.join(TXT_DIR, txt_name)
    if not os.path.exists(txt_path):
        with open(txt_path, "w", encoding="utf-8") as f:
            f.write(pdf_text)
    return txt_path


### 네이버 증권 페이지를 df로 변환하는 함수 

In [24]:
def crawl_all_page_only_to_df() -> pd.DataFrame:
    rows = []
    page = 1
    empty_pages = 0
    done = set()
    
    while True:
        if page > MAX_PAGE:
            break

        nids = get_list_nids(page)

        if not nids:
            empty_pages += 1
            if empty_pages >= 2:
                break
            page += 1
            continue
        
        empty_pages = 0
        for nid in tqdm(nids, desc=f"page {page}"):
            if nid in done:
                continue

            meta = parse_detail_page_raw(nid, debug=True)
            done.add(nid)
            if meta is None:
                continue

            rows.append({
                "nid": meta["nid"],
                "date": meta["date"],
                "org": meta["org"],
                "title": meta["title"],
                "source_raw": meta["source_raw"],
                "page_text": meta["page_text"],
            })

            time.sleep(SLEEP_SEC)

        page += 1

    return pd.DataFrame(rows)


### 개별 pdf text를 하나의 text에 저장

In [38]:
from pathlib import Path
import pandas as pd
import re

def merge_txt_to_csv_with_nid(input_dir: str, output_csv: str):
    input_dir = Path(input_dir)

    # ✅ txt 파일 탐색
    files = sorted(input_dir.glob("*.txt"))
    if len(files) == 0:
        raise FileNotFoundError(
            f".txt 파일을 찾지 못했습니다.\n"
            f"- input_dir: {input_dir}\n"
            f"- 확인: 폴더 경로가 맞는지, 확장자가 정말 .txt인지 확인하세요."
        )

    print(f"발견한 txt 파일 수: {len(files)}")
    print("예시 파일 3개:", [f.name for f in files[:3]])

    rows = []

    for fp in files:
        # ✅ 파일명에서 nid 추출: 마지막 '_' 뒤 숫자
        m = re.search(r"_([0-9]+)$", fp.stem)
        if not m:
            raise ValueError(f"nid 추출 실패(파일명 규칙 확인 필요): {fp.name}")
        nid = m.group(1)

        # ✅ 텍스트 읽기 + 줄바꿈 제거(한 줄화)
        with open(fp, "r", encoding="utf-8", errors="ignore") as f:
            text = f.read()

        # \n 포함 모든 공백을 스페이스 하나로 정리
        text = re.sub(r"\s+", " ", text).strip()

        rows.append({"nid": nid, "text": text})

    df = pd.DataFrame(rows, columns=["nid", "text"])

    # ✅ nid 유일성 체크
    if df["nid"].duplicated().any():
        dup = df.loc[df["nid"].duplicated(keep=False), "nid"].value_counts().head(20)
        raise ValueError(f"중복 nid 발견(상위 20개):\n{dup}")

    df.to_csv(output_csv, index=False, encoding="utf-8-sig")
    print(f"저장 완료: {output_csv} ({len(df)}개 문서)")
    return df


In [ ]:
# 실행
merge_txt_to_csv_with_nid(
    input_dir=r"C:\Users\User\Desktop\kdt\pjt_bok_1\text",
    output_csv="total_pdf_text.csv"
)


발견한 txt 파일 수: 8836
예시 파일 3개: ['2008.04.01_대우증권_12.txt', '2008.04.07_대우증권_11.txt', '2008.04.07_대우증권_33.txt']
저장 완료: total_pdf_text2.csv (8836개 문서)


,nid,text
0,12,2008_04 월간채권투자 채권시장 전망 금융시장 차트북 對應과限界 월간채권투자 4...
1,11,Fixed Income Weekly 2008. 4. 7. #918 Fixed Inc...
2,33,Fixed Income Weekly 2008. 4. 7. #918 Fixed Inc...
3,10,"4월 금통위는 정부-한은간 policy mix 합의의 반영일 수도 예상대로, 올해 ..."
4,9,경기 전망은 비교적 분명… 인플레 명분이 시점 결정할 것 현 금리 레벨이 5월 인하...
...,...,...
8831,10264,-2.0 -2.9 1M 12.29 (월) 1d 5d 3.270 2.870 2.939...
8832,10263,"자료 출처: 연합인포맥스, Refinitiv, 유진투자증권 본 자료는 참고용 자료일..."
8833,10267,2025년 12월 29일 I Global Macro Strategy 하나채권 성장률...
8834,10268,금융투자분석사의 확인 및 중요 공시는 Appendix 참조 받아야 할 $가 사라지면...


In [3]:
# nid type 문자열로 변환
import csv

# CSV 로드
df = pd.read_csv("total_pdf_text2.csv", encoding="utf-8-sig")

# nid를 문자열로 강제 변환
df["nid"] = df["nid"].astype(str)

# 따옴표 유지하여 다시 저장
df.to_csv(
    "total_pdf_text.csv",
    index=False,
    encoding="utf-8-sig",
    quoting=csv.QUOTE_NONNUMERIC
)



Caching the list of root modules, please wait!
(This will only be done once - type '%rehashx' to reset cache!)

This is taking too long, we give up.



### pdf text를 df 로 변환하는 함수

In [ ]:
def crawl_all_pdf_to_df(include_pdf_text: bool = True) -> pd.DataFrame:
    rows = []
    page = 1
    empty_pages = 0
    done = set()

    while True:
        if page > MAX_PAGE:
            break
        
        nids = get_list_nids(page)
        if not nids:
            empty_pages += 1
            if empty_pages >= 2:
                break
            page += 1
            continue

        empty_pages = 0
        for nid in tqdm(nids, desc=f"page {page}"):
            if nid in done:
                continue

            meta = parse_detail_page_raw(nid, debug=True)
            done.add(nid)
            if meta is None or not meta.get("pdf_url"):
                continue

            pdf_path = download_pdf(meta)
            pdf_text = extract_text_from_pdf(pdf_path)

            txt_path = save_pdf_text(meta, pdf_path, pdf_text)

            row = {
                "nid": meta["nid"],
                "date": meta["date"],
                "org": meta["org"],
                "title": meta["title"],
                "pdf_url": meta["pdf_url"],
                "pdf_path": pdf_path,
                "txt_path": txt_path,
            }
            if include_pdf_text:
                row["pdf_text"] = pdf_text

            rows.append(row)
            time.sleep(SLEEP_SEC)

        page += 1

    return pd.DataFrame(rows)


### df를 csv로 저장하는 함수

In [12]:
import csv

def save_df_to_csv(df, path):
    df.to_csv(
        path,
        index=False,
        encoding="utf-8-sig",
        quoting=csv.QUOTE_ALL,
        lineterminator="\n"
    )


### 크롤링한 페이지 정보를 csv로 저장

In [ ]:
df_page = crawl_all_page_only_to_df()

df_page["page_text"] = (
    df_page["page_text"]
    .fillna("")
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

save_df_to_csv(df_page, os.path.join(BASE_DIR, "naver_debenture_page_text.csv"))


page 2: 100%|██████████| 2/2 [00:00<00:00, 330.47it/s]


CSV 저장 완료 → C:\Users\User\Desktop\kdt\pjt_bok_1\naver_debenture_page_text _.csv


### pdf다운로드 및 text저장

In [ ]:
crawl_all_pdf_to_df(include_pdf_text=True)